PRACTICAL 9

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import matplotlib.pyplot as plt
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5WRYGG/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5WRYGG/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5WRYGG/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'I voted 3 for this movie because it looks great as does all of Greenaways output. However it was his usual mix of "art" sex and pretentious crap.I know lots of people like this film but I grew tired of it VERY quickly. It is definitely not for everyone. The ubiquitous McGregor obviously took the part for crediblity\'s sake I guess but he really should not have wasted his time. I hate to consign anyone to pseud\'s corner but please.....!!! On the plus side it IS visually very attractive and I enjoyed the music but could not see it through to the end and I cannot say that for many movies. I usually watch the whole thing but this is unbearable!!'
 b'This film is fun, if your a person who likes a good campy feature film every now and then. By no means is this movie fine cinema, but if you dont take things too seriously, and can laugh at yourself once in a while, Elvira is a good frownbuster.'
 b'If you "get it", it\'s magnificent.<br /><br />If you don\'t, it\'s decent.<br /><br

In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 10,   1, 442, ...,   0,   0,   0],
       [ 11,  20,   7, ...,   0,   0,   0],
       [ 45,  23,  76, ...,   0,   0,   0]])

In [ ]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'I voted 3 for this movie because it looks great as does all of Greenaways output. However it was his usual mix of "art" sex and pretentious crap.I know lots of people like this film but I grew tired of it VERY quickly. It is definitely not for everyone. The ubiquitous McGregor obviously took the part for crediblity\'s sake I guess but he really should not have wasted his time. I hate to consign anyone to pseud\'s corner but please.....!!! On the plus side it IS visually very attractive and I enjoyed the music but could not see it through to the end and I cannot say that for many movies. I usually watch the whole thing but this is unbearable!!'
Round-trip:  i [UNK] 3 for this movie because it looks great as does all of [UNK] [UNK] however it was his usual [UNK] of art sex and [UNK] [UNK] know lots of people like this film but i [UNK] [UNK] of it very quickly it is definitely not for everyone the [UNK] [UNK] obviously took the part for [UNK] [UNK] i guess but he really shoul

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.00045438]


In [ ]:
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0]) 

[0.00045438]


In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
history = model.fit(train_dataset, epochs=3,
                    validation_data=test_dataset,
                    validation_steps=30)

test_loss, test_acc = model.evaluate(test_dataset)

Epoch 1/3
391/391 [==============================] - 737s 2s/step - loss: 0.6399 - accuracy: 0.5750 - val_loss: 0.5144 - val_accuracy: 0.6875
Epoch 2/3
391/391 [==============================] - 727s 2s/step - loss: 0.4407 - accuracy: 0.7824 - val_loss: 0.3900 - val_accuracy: 0.8141
Epoch 3/3
391/391 [==============================] - 162s 414ms/step - loss: 0.3510 - accuracy: 0.8342


In [ ]:
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

Test Loss: 0.3510308861732483
Test Accuracy: 0.8342400193214417
